In [ ]:
#r "nuget: Plotly.NET.Interactive"
#r "nuget: Plotly.NET.ImageExport"

#load "Helpers.fs"
#load "Request.fs"

open Plotly.NET
open Plotly.NET.ImageExport  
open PathfinderAnalysis.Helpers
open Request.Guimard

type DataPrep = {
  ValuesChanceAndKeysDiceRolled: (float * int) list;
  NameResult: string;
}

type Data = {
  Values: float list;
  Keys: int list;
  Name: string;
}

let titleFn = sprintf "Probability of each result for n%s"

let generateChartForDieSize maxDice dieSize =
  let work = 
    [1..maxDice]
    |> List.map (fun nDice -> nDice, requestWork dieSize nDice)

  let data = 
    [Success; SuccessWithComplication; PartialSuccess; PartialSuccessWithComplication; Failure]
    |> List.map (fun result -> 
      List.fold (fun state diceAndResultChances ->
        List.where (fun resultChance -> resultChance.Result = result) (second diceAndResultChances)
        |> List.tryExactlyOne
        |> fun resultChance -> 
          if Option.isNone resultChance then state 
          else List.append state [((Option.get resultChance).Chance, first diceAndResultChances)]
        ) [] work
      |> fun pairs -> { NameResult = string result; ValuesChanceAndKeysDiceRolled = pairs })
      |> List.map (fun preps -> { Name = preps.NameResult; Values = List.map first preps.ValuesChanceAndKeysDiceRolled; Keys = List.map second preps.ValuesChanceAndKeysDiceRolled })

  data
  |> List.map (fun data -> Chart.StackedColumn(
      values = data.Values,
      Keys = data.Keys,
      Name = data.Name,
      MultiText = List.map (fun v -> sprintf "%.0f%%" (100.0*v)) data.Values
    ))
  |> Chart.combine
  |> Chart.withYAxisStyle("chance of result")
  |> Chart.withXAxisStyle("n dice")
  |> Chart.withTitle(titleFn (rollNotation dieSize))

[D4; D6; D8; D10; D12; D20]
|> List.map (generateChartForDieSize 6)
|> Seq.iteri (fun i chart -> Chart.savePNG (path = (sprintf "request-%02i" i)) chart)
